<a href="https://colab.research.google.com/github/danielyangdev/Heart-Disease-Detector/blob/main/Heart_Disease_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **VERSION** **1**

In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from pylab import rcParams
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, ReLU, Dropout

In [ ]:
myData = pd.read_csv('heart.csv')
myData = (myData - np.min(myData))/(np.max(myData) - np.min(myData))

x = myData.drop('target', axis=1)
y = myData['target']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=.2, random_state=2)

In [45]:
model = Sequential()
model.add(Dense(100, input_shape=(13,),activation="relu"))
model.add(Dense(units=64, activation='relu')),
model.add(Dropout(rate=0.2)),
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='categorical_hinge', metrics=['accuracy'])

In [46]:
hist=model.fit(X_train, Y_train,validation_data=(X_test, Y_test), epochs=10)
y_predicted = (model.predict(X_test) >= 0.5)

conf_mat = confusion_matrix(Y_test, y_predicted)
total = sum(sum(conf_mat))
accuracy = (conf_mat[0, 0] + conf_mat[1, 1])/total
print('accuracy : ', accuracy)

Epoch 1/10
26/26 [==============================] - 1s 9ms/step - loss: 0.9471 - accuracy: 0.5902 - val_loss: 0.9315 - val_accuracy: 0.7707
Epoch 2/10
26/26 [==============================] - 0s 3ms/step - loss: 0.8413 - accuracy: 0.7805 - val_loss: 0.8105 - val_accuracy: 0.8098
Epoch 3/10
26/26 [==============================] - 0s 4ms/step - loss: 0.7471 - accuracy: 0.7963 - val_loss: 0.7412 - val_accuracy: 0.8098
Epoch 4/10
26/26 [==============================] - 0s 3ms/step - loss: 0.7045 - accuracy: 0.8012 - val_loss: 0.7189 - val_accuracy: 0.8146
Epoch 5/10
26/26 [==============================] - 0s 4ms/step - loss: 0.6809 - accuracy: 0.8305 - val_loss: 0.7075 - val_accuracy: 0.8244
Epoch 6/10
26/26 [==============================] - 0s 3ms/step - loss: 0.6705 - accuracy: 0.8341 - val_loss: 0.7009 - val_accuracy: 0.8244
Epoch 7/10
26/26 [==============================] - 0s 3ms/step - loss: 0.6629 - accuracy: 0.8341 - val_loss: 0.6966 - val_accuracy: 0.8195
Epoch 8/10
26/26 [==

### **VERSION** **2** **(wrong)**

In [8]:
import tensorflow as tf
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Other/CAIS++/F23_Curriculum/Heart_Disease_Detector/kaggle.json"
!kaggle datasets download -d johnsmith88/heart-disease-dataset

In [ ]:
heart_data = pd.read_csv('heart.csv')
heart_data.info()
heart_data.head()
heart_data.describe()

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __getitem__(self, index):
        row = self.dataframe.iloc[index].to_numpy()
        features = row[1:]
        label = row[0]
        return features, label

    def __len__(self):
        return len(self.dataframe)


df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                  columns=['label', 'feature_0', 'feature_1'])
data = heart_data(dataframe=df)
dataloader = DataLoader(data)
for sample in dataloader:
    print(sample)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch_size = 8

#train_data = heart_data(root='./data', train=True, transform=transforms.ToTensor(), download=True)
#test_data = heart_data(root='./data', train=False, transform=transforms.ToTensor(), download=True)

#train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle=True)
#test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle=False)

In [13]:
input_size = 784
num_classes = 3
num_epochs = 10

class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.relu = nn.ReLU()
    self.input_layer =  nn.Linear(input_size, 64)
    self.hidden1 = nn.Linear(64, 32)
    self.output_layer = nn.Linear(32, 10)

  def forward(self, sample):
    out = self.input_layer(sample)
    out = self.relu(out)
    out = self.hidden1(out)
    out = self.relu(out)
    out = self.output_layer(out)
    return out

nn_model = NN(input_size, num_classes)

In [14]:
loss_function = nn.CrossEntropyLoss()
oprimizer = torch.optim.Adam(nn_model.parameters(), lr = 0.0001)

In [ ]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    #images = images.reshape(-1,28*28).to(device)
    labels = labels.to(device)

    output = nn_model.forward(images)
    loss = loss_function(output, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{total_steps}, loss = {loss.item():.4f}')

In [ ]:
with torch.no_grad():
  accuracy_arr = []
  loss_arr = []
  correct_predict_num = 0
  sample_num = 0
  for images, labels in test_loader:
    images = images.reshape(batch_size, -1).to(device)
    labels = labels.to(device)
    output = nn_model(images)

    loss, predictions = torch.max(output, 1)
    sample_num += labels.shape[0]
    correct_predict_num += (predictions == labels).sum().item()

    loss = loss_function(output, labels)
    accuracy =  correct_predict_num / sample_num
    accuracy_arr.append(accuracy)
    loss_arr.append(loss)

print(f'Average Accuracy: {sum(accuracy_arr)/len(accuracy_arr)}')
print(f'Average Cost/Loss: {sum(loss_arr)/len(loss_arr)}')

In [ ]:
sample_num = 1

images, labels = next(iter(train_loader))
test_images = images.reshape(-1, 28*28).to(device)
test_labels = labels.to(device)

prediction_outputs = nn_model(test_images[sample_num])  # outputted probabilities vector
print("Prediction output: ", prediction_outputs)

predicted_digit = torch.argmax(prediction_outputs) # pick the class with highest probability --> final prediction
print("Predicted digit: ", predicted_digit.item()) # print predicted classification

# Show actual input image
plt.imshow(images[sample_num][0], cmap="gray")
plt.show()